In [1]:
import getpass
import os

if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")

Enter your OpenAI API key: ··········


In [2]:
!pip install -qU langchain-openai langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 408.0/408.0 kB 27.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.9/296.9 kB 22.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.9/386.9 kB 25.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 45.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.2/325.2 kB 21.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.5/144.5 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/

In [3]:
from langchain_openai import ChatOpenAI

In [4]:
model = ChatOpenAI(model="gpt-4o")

In [5]:
model.invoke('Cuanto es 2 🦜 9').content

'Parece que has usado un emoji de loro (🦜) en lugar de un símbolo matemático. Si quieres realizar una operación matemática, por favor aclara qué operación deseas hacer (por ejemplo, suma, resta, multiplicación, etc.).'

In [6]:
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate

examples = [{'input': '2 🦜 2', 'output': '4'},
            {'input': '2 🦜 3', 'output': '5'}]

In [7]:
example_prompt = ChatPromptTemplate(
    [('human', '{input}'),
     ('ai', '{output}')]
)

In [8]:
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples
)

In [9]:
print(few_shot_prompt.invoke({}).to_messages)

<bound method ChatPromptValue.to_messages of ChatPromptValue(messages=[HumanMessage(content='2 🦜 2', additional_kwargs={}, response_metadata={}), AIMessage(content='4', additional_kwargs={}, response_metadata={}), HumanMessage(content='2 🦜 3', additional_kwargs={}, response_metadata={}), AIMessage(content='5', additional_kwargs={}, response_metadata={})])>


In [10]:
main_prompt = ChatPromptTemplate.from_messages(
    [('system', 'Eres un mago de las matematicas.'),
     few_shot_prompt,
     ('human', '{input}')
     ]
)

In [11]:
chain = main_prompt | model

In [12]:
chain.invoke({'input': 'Cuanto es 2 🦜 9'}).content

'El símbolo 🦜 parece estar representando una operación matemática básica, como la suma. Si es así, entonces \\(2 🦜 9\\) sería igual a \\(2 + 9 = 11\\). Si este símbolo representa otra operación, por favor aclara su significado.'



Few-shot Prompting es una técnica que permite a los modelos de lenguaje aprender a responder de manera más precisa utilizando ejemplos concretos dentro del mismo prompt. Los ejemplos ayudan a guiar las respuestas del modelo, mejorando su relevancia. Hay dos formas de implementar esta técnica: usando ejemplos fijos o dinámicos, lo que permite adaptar las interacciones según la entrada.

Conceptos Principales:

Few-shot Prompting: Utiliza ejemplos dentro del prompt para enseñar al modelo cómo debe comportarse. Los ejemplos actúan como guías para que el modelo aprenda a responder de manera precisa.
Ejemplo: Si queremos que el modelo interprete emojis como operaciones matemáticas, proporcionamos ejemplos como "2 🦜 2 = 4" para que el modelo aprenda la relación.
Ejemplos Fijos: Conjunto estático de ejemplos que se utilizan en cada interacción, independientemente de la entrada del usuario.
Ejemplo: Siempre usar "2 🦜 2 = 4" y "2 🦜 3 = 5" para que el modelo entienda cómo operar con emojis.
Ejemplos Dinámicos: Los ejemplos se seleccionan según la similitud semántica entre la entrada del usuario y los ejemplos almacenados.
Ejemplo: Si el usuario pregunta "¿cuánto es 5 emoji 5?", se selecciona un ejemplo similar para que el modelo aprenda a sumar usando emojis.
Prompt Template: Pasa ejemplos al modelo. Cada ejemplo tiene una entrada (input) y una salida (output) esperada, generalmente estructurados en formato de diccionario.
Ejemplo: Un diccionario con {input: "2 🦜 2", output: "4"} enseña al modelo que el emoji representa una operación matemática.
Chains: Permite integrar múltiples pasos en una cadena, combinando el modelo con ejemplos y roles para crear interacciones más complejas.
Ejemplo: Encadenar un prompt que define al modelo como "un mago de las matemáticas" junto con ejemplos para que se comporte de acuerdo a la expectativa.






